Primarily adapted from Sam Maurer's UAL code

In [1]:
import os; os.chdir('../')

In [2]:
import pandas as pd
import numpy as np
import orca
from scripts import datasources, models
from urbansim.utils import misc

### Create empty units

In [3]:
def _ual_create_empty_units(buildings):
    df = pd.DataFrame({
        'unit_residential_price': 0,
        'unit_residential_rent': 0,
        'num_units': 1,
        'building_id': np.repeat(buildings.index.values,
                                 buildings.residential_units.values.astype(int)),
        # counter of the units in a building
        'unit_num': np.concatenate([np.arange(i) for i in \
                                    buildings.residential_units.values.astype(int)]),
        'tenure': -1
    }).sort_values(by=['building_id', 'unit_num']).reset_index(drop=True)
    df.index.name = 'unit_id'
    return df

### Initialize residential units

In [4]:
@orca.table('residential_units', cache=True)
def residential_units(buildings):
    return _ual_create_empty_units(buildings)

### Subtract occupied units (\*\*\**cannot be called until households have been assigned to units*\*\*\*)

In [6]:
# @orca.column('residential_units', 'vacant_units')
# def vacant_units(residential_units, households):
#     return residential_units.num_units.sub(
#             households.unit_id[households.unit_id != -1].value_counts(), fill_value=0)

### Big merge to get data we need for tenure imputation

In [5]:
units = orca.merge_tables('residential_units', [
    'buildings', 'residential_units', 'parcels', 'geog_mappings', 'bldg_type_tenure_by_block_group'])

### Assign tenure to units

In [6]:
# if building type == 1, sample tenure from rate of single-family ownership
units.loc[units['building_type_id'] == 1, 'tenure'] = [
    1 if x else 2 for x in 
    np.random.random(len(units[units['building_type_id'] == 1])) <= units.loc[units['building_type_id'] == 1, 'sf_o']]

In [7]:
# if building type > 1, sample_tenure from rate of multi-family ownership
units.loc[units['building_type_id'] > 1, 'tenure'] = [
    1 if x else 2 for x in 
    np.random.random(len(units[units['building_type_id'] > 1])) <= units.loc[units['building_type_id'] > 1, 'mf_o']]

In [8]:
# if building type > 1 and redfin_home_type == 'Condo/Coop', tenure = 1
units.loc[(units['redfin_home_type'] == 'Condo/Coop') & (units['building_type_id'] > 1), 'tenure'] = 1 

In [11]:
units[orca.get_table('residential_units').columns].to_csv('./data/units_w_tenure.csv')